# ENEM 2019: Previsão das notas de Matemática a partir de dados socioeconômicos
## Treinando um XGBoost


Esse é o mesmo projeto que já realizei em python. A novidade aqui é que usaremos a linguagem R para treinar um modelo XGBoost. Por isso, nesse notebook, não faremos a modelagem completa (e.g. conjunto de validação e teste, refinamento de hiperparâmetros, etc). A modelagem completa pode ser vista [aqui](https://www.kaggle.com/hurzzz/enem-2019).

In [1]:
#carregando as bibliotecas
library(tidyverse)
library(caret) #para o train/test split
library(xgboost)

#carregando os dados
X <- read.csv('X_final.csv')
y  <- read.csv('y_final.csv')

full_data <- cbind(X, y)

str(full_data)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: lattice

Attaching package: 'caret'

The following object is masked from 'package:purrr':

    lift



'data.frame':	3633513 obs. of  6 variables:
 $ Q005      : int  7 5 3 5 4 4 4 4 3 3 ...
 $ Q006      : int  1 1 2 2 2 1 1 1 1 2 ...
 $ TP_SEXO   : int  0 1 1 0 1 1 0 1 0 1 ...
 $ max_escol : int  4 4 4 2 1 1 3 4 4 3 ...
 $ perCapita : int  1 1 3 2 2 1 1 1 1 3 ...
 $ NU_NOTA_MT: num  369 416 572 605 582 ...


Dividindo os dados em conjunto de treinamento e validação:

In [30]:
#train/test split
trainIndex <- createDataPartition(full_data$NU_NOTA_MT, p = .9, 
                                  list = FALSE, 
                                  times = 1)

full_data_train <- full_data[ trainIndex,]
full_data_test  <- full_data[-trainIndex,]

X_train <- X[ trainIndex,]
y_train <- y[ trainIndex,]

X_test  <- X[-trainIndex,]
y_test  <- y[-trainIndex,]

In [ ]:
#xgbcv <- xgb.cv( params = {}, data = X_train, nrounds = 500, nfold = 5, showsd = T, stratified = T, print_every_n = 40, early_stopping_rounds = 10, maximize = F)

In [15]:
#parâmetros ótimos encontrados por bayesian search no projeto feito em python
params <- list(booster = "gbtree"
              , objective = "reg:squarederror"
              , colsample_bytree = 0.7971305
              , eta = 0.122620
              , reg_alpha = 0.282408
              , reg_lambda = 0.84137
              , eval_metric = 'rmse'
              , min_child_weight = 275)

In [31]:
#colocando nossos dados no formato aceito pela biblioteca xgboost
xgb_train = xgb.DMatrix(data = as.matrix(X_train), label = as.matrix(y_train))
xgb_test = xgb.DMatrix(data = as.matrix(X_test), label = as.matrix(y_test))

In [32]:
#treinando o modelo
model <- xgb.train(params = params
                 , data = xgb_train
                 , nrounds = 173
                 , verbose = 1
                 , print_every_n = 2
                )

In [34]:
#fazendo a previsão no conjunto de validação
pred_y = predict(model, xgb_test)

#avaliando o modelo
mse = mean((y_test - pred_y)^2)
mae = caret::MAE(y_test, pred_y)
rmse = caret::RMSE(y_test, pred_y)

cat("MSE: ", mse, "MAE: ", mae, " RMSE: ", rmse)

MSE:  8686.489 MAE:  75.23165  RMSE:  93.20133

O RMSE no conjunto de teste (exemplo nunca vistos antes pelo modelo) foi de aproximadamente 93, compatível com o valor encontrado no projeto desenvolvido em python